In [1]:
import math
import numpy as np
import torch
import torchsde
import gc  # Garbage Collector

import torch
from torch import nn
import time

import os
import torch
import numpy as np
from torch_geometric.datasets import Planetoid, Coauthor
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops, degree
import os
import torch_geometric.transforms as T

from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import GATConv

from torchdiffeq import odeint
from ogb.nodeproppred import PygNodePropPredDataset
from dgl.data import CoauthorCSDataset, AmazonCoBuyComputerDataset, AmazonCoBuyPhotoDataset
import copy
# from heterophilic import WebKB

/scratch/rsb63/miniconda3/envs/gn-sde/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scratch/rsb63/miniconda3/envs/gn-sde/lib/python3.9/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /scratch/rsb63/miniconda3/envs/gn-sde/lib/python3.9/site-packages/torch_sparse/_diag_cuda.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowIlEET_S2_S2_b
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
from gn_sde import LatentGraphSDE
from bayesian_gcn import BayesianGCNConv


In [3]:
def load_data(dataset_name, root_path='dataset', transform=None):
    create_results_folder(dataset_name)

    if dataset_name.startswith('ogbn'):
        dataset = PygNodePropPredDataset(name=dataset_name, root=root_path, transform=transform)
    elif dataset_name == 'CoauthorCS':
        dataset = CoauthorCSDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    elif dataset_name == 'Computer':
        dataset = AmazonCoBuyComputerDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    elif dataset_name == 'Photo':
        dataset = AmazonCoBuyPhotoDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    # elif dataset_name == ['cornell', 'texas', 'wisconsin']:
    #     dataset = WebKB(root=path, name=ds, transform=T.NormalizeFeatures())
    else:
        path = os.path.join(os.getcwd(), root_path, dataset_name)
        dataset = Planetoid(path, dataset_name, transform=transform)

    return dataset

def set_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)

def create_results_folder(data_name):
    folder_name = f'results/{data_name.lower()}'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder created: {folder_name}")
    else:
        print(f"Folder already exists: {folder_name}")



In [4]:
def preprocess_data(data, dataset_name):
    if (dataset_name == 'Computer') or (dataset_name == 'CoauthorCS') or (dataset_name == 'Photo'):
        g = data
        X = g.ndata['feat']
        Y = g.ndata['label']
        # edge_index = torch.tensor(g.edges(), dtype=torch.long)
        src, dst = g.edges()
        edge_index = torch.stack([src, dst], dim=0)
        # edge_index = to_undirected(edge_index)  # Convert to undirected if needed

        num_nodes = g.num_nodes()
        train_mask, val_mask,  test_mask = create_masks(num_nodes)

        # Wrap the CoauthorCS dataset into a PyG Data object
        data = Data(x=X, y=Y, edge_index=edge_index, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

    
    edge_index = data.edge_index
    # Compute normalization
    deg = degree(edge_index[0], dtype=torch.float)
    norm = torch.pow(deg, -0.5)
    norm[torch.isinf(norm)] = 0
    
    X = data.x
    Y = data.y.squeeze()

    if dataset_name.startswith('ogbn'):
        split_idx = data.get_idx_split()
        train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

        train_mask[split_idx['train']] = True
        val_mask[split_idx['valid']] = True
        test_mask[split_idx['test']] = True
        
    else:
        train_mask = data.train_mask
        val_mask = data.val_mask
        test_mask = data.test_mask

    return X, Y, train_mask, val_mask, test_mask, edge_index, norm

def create_masks(num_nodes, train_percent=0.8, val_percent=0.1):
    if train_percent + val_percent > 1.0:
        raise ValueError("Sum of training and validation percentages cannot exceed 100%.")

    all_indices = torch.randperm(num_nodes)
    train_size = int(num_nodes * train_percent)
    val_size = int(num_nodes * val_percent)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[all_indices[:train_size]] = True
    val_mask[all_indices[train_size:train_size + val_size]] = True
    test_mask[all_indices[train_size + val_size:]] = True

    return train_mask, val_mask, test_mask




Get Random Seed and Device

In [5]:
device = set_device()
print('device', device)
set_seed(12345)

device cuda:0


Load Dataset

In [6]:
# For example, to load the 'Cora' dataset:
data_name = 'Cora'
dataset = load_data(data_name)

# Assuming you have already loaded a dataset into the variable 'dataset'
data = dataset[0].to(device)  # Move data to the specified device
X, Y, train_mask, val_mask, test_mask, edge_index, norm = preprocess_data(data, data_name)
num_feats = X.shape[1]
# batch_size = 64
# num_feats = batch_size
n_classes = len(set(np.array(Y.cpu())))

Folder already exists: results/cora


### Models

In [7]:

class MyGCN(torch.nn.Module):
    def __init__(self, edge_index, in_feats, out_feats, activation=None, dropout=0):
        super(MyGCN, self).__init__()
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.conv1 = GCNConv(in_feats, out_feats)

    def forward(self, x):
        x = self.conv1(x, self.edge_index)
        if self.activation is not None:
            x = self.activation(x)
        if self.training:  # Apply dropout only during training
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x


### GN-SDE

In [8]:
n_dimension = 64
gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)
gnn_f = nn.Sequential(
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2)
).to(device)
# Complete gcn_out
gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)

gn_sde = LatentGraphSDE(in_net=gcn_in, drift_net=gnn_f, out_net=gcn_out).to(device)



self.embeddings {'0': 0, '0.0001': 0, '0.0002': 0, '0.0003': 0, '0.0004': 0, '0.0005': 0, '0.0006': 0, '0.0007': 0, '0.0008': 0, '0.0009': 0, '0.001': 0}
using file gn_sde.py
sigma 1.0
rtol 0.01
t1 0.001


In [9]:
def initialize_graph_neural_sde(device=device, n_dimension=n_dimension, num_classes=n_classes):

    gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)

    gnn_f = nn.Sequential(
    MyGCN(edge_index=edge_index, in_feats=n_dimension +1, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2), # The +1 is for the time.
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=None, dropout=0.2)
    ).to(device)


    # Complete gcn_out
    gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=num_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)
        


    # Initialize the Graph Neural SDE model
    graph_neural_sde = LatentGraphSDE(drift_net=gnn_f, in_net=gcn_in, out_net=gcn_out, sde_output_dim=n_dimension).to(device)

    return graph_neural_sde

### GN-ODE

In [10]:

class GraphNeuralODE(torch.nn.Module):
    def __init__(self, ode_func, in_net=None, out_net=None, method='rk4', atol=1e-3, rtol=1e-4):
        super(GraphNeuralODE, self).__init__()
        self.in_net = in_net
        self.projection_net = out_net

        self.ode_func = ode_func
        self.method = method
        
        self.atol = atol
        self.rtol = rtol

    def f(self, t, y):
        return self.ode_func(y)

    def forward(self, x):
        x = self.in_net(x).to(device)
        t = torch.tensor([0, 1]).float().to(device)  # Define the time steps
        t = t.type_as(x)
        # t_span = torch.linspace(t[0], t[1], steps=4)  # Adjust the number of steps as needed
        x = odeint(self.f, x.to(device), t.to(device) , method=self.method, atol=self.atol, rtol=self.rtol)
        ode_output_time_1 = x[-1].to(device)
        x = self.projection_net(ode_output_time_1)
        return x


# we wrap all of this Neural ODE initializer
def initialize_graph_neural_ode():

    # Initialize GCN layers
    gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)
    
    # Define and initialize ODE function
    
    gde_func = nn.Sequential(
        MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2),
        MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=None, dropout=0.2)
    ).to(device)


    # Output Layer
    gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)
        
    # Initialize Neural ODE
    neural_ode = GraphNeuralODE(in_net=gcn_in, ode_func=gde_func, out_net=gcn_out).to(device)

    return neural_ode



### GCN

In [11]:
class MyGCN2(torch.nn.Module):
    def __init__(self, edge_index, in_feats=num_feats, out_feats=n_classes, activation=None, dropout=0):
        super(MyGCN2, self).__init__()
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.conv1 = GCNConv(in_feats, n_dimension)
        # self.conv2 = GCNConv(n_dimension, n_dimension)
        self.conv4 = GCNConv(n_dimension, out_feats)

    def forward(self, x):
        x = self.conv1(x, self.edge_index)
        if self.activation is not None:
            x = self.activation(x)
        if self.training:
            x = F.dropout(x, p=0.4)
            
        # x = self.conv2(x, self.edge_index)
        # x = self.activation(x)
        # if self.training:
        #     x = F.dropout(x, p=self.dropout)

        x = self.conv4(x, self.edge_index)
        return F.softmax(x, dim=1)
    
    
gcn_model = MyGCN2(edge_index=edge_index, in_feats=num_feats, out_feats=n_classes, activation=F.softplus, dropout=0.2).to(device)
    

In [12]:
def initialize_gcn_model(device=device, num_feats=num_feats, n_dimension=n_dimension, dropout=0.2):
    model = MyGCN2(edge_index=edge_index, in_feats=num_feats, out_feats=n_classes, activation=F.softplus, dropout=dropout).to(device)
    return model

gcn_model = initialize_gcn_model()

In [13]:
gcn_model

MyGCN2(
  (conv1): GCNConv(1433, 64)
  (conv4): GCNConv(64, 7)
)

### Bayesian GCN

In [14]:
class BayesianGCN(torch.nn.Module):
    def __init__(self, edge_index, in_feats=num_feats, out_feats=n_classes, activation=None, dropout=0):
        super(BayesianGCN, self).__init__()
        """
        Create a Bayesian Graph Convolutional Network with two layers.
        
        Parameters:
            g: Graph structure (e.g., from DGL or PyTorch Geometric)
            num_features: Number of input features
            hidden_units: Number of units in the hidden layer
            num_classes: Number of output classes (or units in the output layer)
        """
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.bayes_conv1 = BayesianGCNConv(in_feats, n_dimension, prior_mean = 0.0, prior_variance = 1.0) 
        # self.bayes_conv2 = BayesianGCNConv(n_dimension, n_dimension, prior_mean = 0.0, prior_variance = 1.0) 
        self.bayes_conv3 = BayesianGCNConv(n_dimension, out_feats, prior_mean = 0.0, prior_variance = 1.0)

    def forward(self, data, return_kl=True):
        x = data
        x, kl1 = self.bayes_conv1(x, self.edge_index, return_kl=return_kl)
        if self.activation is not None:
            x = self.activation(x)
        # x, kl2 = self.bayes_conv2(x, self.edge_index, return_kl=return_kl)
        # if self.activation is not None:
        #     x = self.activation(x)
        x = F.dropout(x, p=self.dropout)
        x, kl3 = self.bayes_conv3(x, self.edge_index, return_kl=return_kl)
        
        if return_kl:
            total_kl = kl1 + kl3
            return F.softmax(x, dim=1), total_kl
        return F.softmax(x, dim=1)    
        

In [15]:
def initialize_bayesian_gcn_model(device=device, num_feats=num_feats, num_classes=n_classes, dropout=0.2):
    model = BayesianGCN(edge_index=edge_index, in_feats=num_feats, out_feats=num_classes, activation=F.softplus, dropout=dropout).to(device)
    return model

bayes_gcn_model = initialize_bayesian_gcn_model()

### Ensemble of GCN

In [16]:
# Step 1: Train individual models
number_of_ensemble = 5
gcn_models = [initialize_gcn_model() for _ in range(number_of_ensemble)]

In [17]:
def mc_sample_stats(model, data, n_samples=100):
    samples = []
    samples_entropy = []
    for _ in range(n_samples):
        output_probability, _ = model(data)
        sample = output_probability
        samples.append(sample)
        sample_entropy = entropy_fn(sample)
        samples_entropy.append(sample_entropy)
    
    samples_tensor = torch.stack(samples) # to keep it consistant we keep the samples mean and var as torch tensors.
    var = torch.var(samples_tensor, dim=0, unbiased=True).sum(dim=1)  # Sum across features for each sample
    mean = torch.mean(samples_tensor, dim=0)
    mean_entropy = torch.mean(samples_tensor, dim=0)
    
    return var, mean, mean_entropy
    
    
def evaluate_stochastic_model(model, data, n_samples=100, entropy_threshold=np.inf, mask_data=test_mask):
    model.eval()
    with torch.no_grad():
        _, samples_mean, samples_entropy = mc_sample_stats(model, X, n_samples=n_samples)
        acc, num_of_predictions = accuracy_with_entropy_threshold(samples_mean, samples_entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)
    return acc, num_of_predictions



def entropy_fn(prediction_probabilities):
    eps = 1e-9
    p_clipped = torch.clamp(prediction_probabilities, eps, 1 - eps)
    entropy = -(p_clipped * torch.log(p_clipped)).sum(dim=1)
    return entropy
        

def accuracy_with_entropy_threshold(prediction_probabilities, entropy, labels, entropy_threshold, mask_data):
    # Only predictions if the entropy is less than the given threshold
    predictions_probabilities = prediction_probabilities[mask_data]
    labels = labels[mask_data]
    entropy = entropy_fn(prediction_probabilities)
    entropy = entropy[mask_data]
    mask = (entropy <= entropy_threshold) # True if the entropy of the prediction is less or equal to the threshold
    predictions_bellow_entropy = predictions_probabilities[mask].to('cpu')
    labels = labels[mask].to('cpu') 
    _, pred = predictions_bellow_entropy.max(dim=1)
    correct = float(pred.eq(labels).sum().item())
    num_of_predictions = labels.size(0)
    if num_of_predictions > 0:
        acc = correct / num_of_predictions
    else:
        acc = None  # to avoid division by zero
    return acc, num_of_predictions


def evaluate_with_entropy_threshold(model, data, entropy_threshold=np.inf, mask_data=test_mask):
    model.eval()
    with torch.no_grad():
        prediction = model(X)
        entropy = entropy_fn(prediction)
        acc, num_of_predictions = accuracy_with_entropy_threshold(prediction, entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)
    return acc, num_of_predictions



def ensemble_inference(models, data, entropy_threshold=np.inf, mask_data=test_mask):
    X, Y, _, test_mask = data
    predictions = []
    # Generate predictions from each model
    for model in models:
        model.eval()
        output = model(X)
        predictions.append(output.data.cpu())

    # Aggregate predictions (e.g., using mean)
    ensemble_prediction = torch.mean(torch.stack(predictions), dim=0).to(device)
    # Calculate uncertainty (e.g., using entropy)
    entropies = [entropy_fn(prediction) for prediction in predictions]
    mean_entropy = torch.mean(torch.stack(entropies), dim=0).to(device)
    # variance_entropy = torch.var(torch.stack(entropies), dim=0)

    acc, num_of_predictions = accuracy_with_entropy_threshold(ensemble_prediction, mean_entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)

    return acc, num_of_predictions


def evaluate_model_type(model, data, model_type, entropy_threshold, num_sample=10, mask_data=test_mask):
    if model_type =='deterministic': 
        return evaluate_with_entropy_threshold(model, data, entropy_threshold=entropy_threshold, mask_data=mask_data)
    elif model_type =='stochastic':
        return evaluate_stochastic_model(model, data, n_samples=num_sample, entropy_threshold=entropy_threshold, mask_data=mask_data)
    elif model_type == 'ensemble':
        return ensemble_inference(model, data, entropy_threshold=entropy_threshold, mask_data=mask_data)
    else:
        raise ValueError(f"Invalid model_type: {model_type}. Supported values are 'stochastic', 'deterministic', or 'ensemble'.")

In [18]:
class LinearScheduler(object):
    def __init__(self, iters, maxval=1.00):
        self._iters = max(1, iters)
        self._val = maxval / self._iters
        self._maxval = maxval

    def step(self):
        self._val = min(self._maxval, self._val + self._maxval / self._iters)

    @property
    def val(self):
        return self._val

def accuracy_fn(y_pred, y_true):

    # Convert raw scores to probabilities using softmax if necessary
    if y_pred.shape[1] > 1:  # Assuming class scores are in the second dimension
        y_pred = torch.softmax(y_pred, dim=1)
        predictions = torch.argmax(y_pred, dim=1)
    else:
        # For binary classification, apply sigmoid and threshold at 0.5
        predictions = torch.sigmoid(y_pred) >= 0.5

    correct = torch.eq(predictions, y_true).sum().item()
    total = y_true.shape[0]
    return correct / total

criterion = torch.nn.CrossEntropyLoss()
data = (X, Y, train_mask, test_mask)  # Assuming these are defined
kl_scheduler = LinearScheduler(iters=200)

In [19]:

def train_model(model, criterion, data, steps, verbose_step, kl_scheduler=kl_scheduler, accuracy_func=accuracy_fn, is_stochastic=False):
    optimizer = torch.optim.Adam(model.parameters())
    best_model = None
    best_val_acc = 0.0
    
    for step in range(steps+1):
        model.train()

        if is_stochastic:
            outputs, kl_div = model(X)
            ce_loss = criterion(outputs[train_mask], Y[train_mask])
            total_loss = ce_loss + (kl_div * kl_scheduler.val if kl_scheduler else 0)
        else:
            outputs = model(X)
            total_loss = criterion(outputs[train_mask], Y[train_mask])

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            model.eval()
            if is_stochastic:
                outputs, _ = model(X)
            else:
                outputs = model(X)
                
            train_loss = total_loss.item()
            train_acc = accuracy_func(outputs[train_mask], Y[train_mask])
            test_acc = accuracy_func(outputs[test_mask], Y[test_mask]) 
            val_acc = accuracy_func(outputs[val_mask], Y[val_mask])
            
            # Save the best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model = copy.deepcopy(model.state_dict())
                print(f'[{step}], Best Model with a Train Accuracy: {train_acc:.3f}, Val Accuracy: {val_acc:.3f}, Test Accuracy: {test_acc:.3f}')
        

        if step % verbose_step == 0:
            print(f'[{step}], Loss: {train_loss:.3f}, Train Accuracy: {train_acc:.3f}, Val Accuracy: {val_acc:.3f}, Test Accuracy: {test_acc:.3f}')
            
    # Load the best model after training
    if best_model is not None:
        model.load_state_dict(best_model)
        
    return model



In [20]:

def evaluate_uncertainty_quantification(model_initializer, num_experiments, model_name, entropy_thresholds, data, model_type, is_stochastic):
    all_accuracies = {str(entropy_threshold): [] for entropy_threshold in entropy_thresholds}

    for experiment_number in range(num_experiments):
        print("experiment_number", experiment_number)

        # Ensure the model is on the correct device (GPU if available) for training
        model = model_initializer()
        model.to(device)

        # Train model
        model = train_model(model, criterion, data, steps=2000, verbose_step=100, kl_scheduler=kl_scheduler, accuracy_func=accuracy_fn, is_stochastic=is_stochastic)

        for entropy_threshold in entropy_thresholds:
            print('~' * 10 + f' Model: {model_name}, Entropy Threshold: {entropy_threshold}' + '~' * 10)
            test_accuracy, num_pred = evaluate_model_type(model, data, model_type=model_type, entropy_threshold=entropy_threshold) 
            print(f'Entropy Threshold: {entropy_threshold}', 'Number of Predictions', num_pred, 'Test accuracy:', test_accuracy)
            all_accuracies[str(entropy_threshold)].append(test_accuracy)

        # Move model to CPU to free up GPU memory after evaluation (optional, depends on memory constraints)
        model.to('cpu')

        # Clear cached memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Explicitly collect garbage
        gc.collect()

    return all_accuracies


def train_ensemble(steps=2000, verbose_step=100, kl_scheduler=None, accuracy_func=accuracy_fn, is_stochastic=False):    
    gcn_models = [initialize_gcn_model().to(device) for _ in range(number_of_ensemble)]
    for model in gcn_models:
        model = train_model(model, criterion, data, steps, verbose_step, kl_scheduler=kl_scheduler, accuracy_func=accuracy_func, is_stochastic=is_stochastic)
    ensemble_model = gcn_models

    acc, num_pred = ensemble_inference(ensemble_model, data)
    return ensemble_model, acc



def evaluate_uncertainty_quantification_ensemble(num_experiments, model_name, entropy_thresholds, data):
    all_accuracies = {str(entropy_threshold): [] for entropy_threshold in entropy_thresholds}

    for experiment_number in range(num_experiments):
        print("experiment_number", experiment_number)
        # Get data with the training percentages

        ensemble_model, acc = train_ensemble()
        # include model
        for entropy_threshold in entropy_thresholds:
            print('~' * 10 + f' Model: {model_name}, Entropy Threshold: {entropy_threshold}' + '~' * 10)
            test_accuracy, num_pred = ensemble_inference(ensemble_model, data, entropy_threshold=entropy_threshold) 
            print(f'Entropy Threshold: {entropy_threshold}', 'Number of Predictions', num_pred, 'Test accuracy:', test_accuracy)
            # Assuming 'test_accuracy' is the accuracy of the model on the test set
            all_accuracies[str(entropy_threshold)].append(test_accuracy)
            
        
    # Move model to CPU to free up GPU memory after evaluation (optional, depends on memory constraints)
    for model in ensemble_model:
        model.to('cpu')

    # Clear cached memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Explicitly collect garbage
    gc.collect()

    return all_accuracies

In [21]:

import json
def save_results(all_accuracies, model_name, results_path):
    filename = f'{results_path}/accuracies_{model_name}.json'

    # Check if the file already exists
    counter = 1
    while os.path.exists(filename):
        filename = f'{results_path}/accuracies_{model_name}_(%d).json' % counter
        counter += 1

    with open(filename, 'w') as fp:
        json.dump(all_accuracies, fp, indent=4)


### Train Models

In [22]:
num_experiments = 1
entropy_results_path = f'results/{data_name.lower()}' 
entropy_thresholds = [np.inf, 2, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

In [23]:
criterion = torch.nn.CrossEntropyLoss()
data = (X, Y, train_mask, test_mask)  
kl_scheduler = LinearScheduler(iters=200)

### Graph Neural SDE

In [24]:
model_sde_name = 'graph_neural_sde'
graph_neural_sde = initialize_graph_neural_sde

acc_sde = evaluate_uncertainty_quantification(graph_neural_sde, num_experiments, model_sde_name, entropy_thresholds, data, model_type='stochastic', is_stochastic=True)
save_results(acc_sde, model_sde_name, entropy_results_path)


experiment_number 0
self.embeddings {'0': 0, '0.0001': 0, '0.0002': 0, '0.0003': 0, '0.0004': 0, '0.0005': 0, '0.0006': 0, '0.0007': 0, '0.0008': 0, '0.0009': 0, '0.001': 0}
using file gn_sde.py
sigma 1.0
rtol 0.01
t1 0.001


/tmp/ipykernel_3043090/2244855087.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation(x)


[0], Best Model with a Train Accuracy: 0.293, Val Accuracy: 0.228, Test Accuracy: 0.203
[0], Loss: 1.945, Train Accuracy: 0.293, Val Accuracy: 0.228, Test Accuracy: 0.203
[1], Best Model with a Train Accuracy: 0.529, Val Accuracy: 0.306, Test Accuracy: 0.295
[2], Best Model with a Train Accuracy: 0.607, Val Accuracy: 0.402, Test Accuracy: 0.389
[3], Best Model with a Train Accuracy: 0.750, Val Accuracy: 0.476, Test Accuracy: 0.444
[4], Best Model with a Train Accuracy: 0.814, Val Accuracy: 0.542, Test Accuracy: 0.530
[5], Best Model with a Train Accuracy: 0.871, Val Accuracy: 0.578, Test Accuracy: 0.570
[6], Best Model with a Train Accuracy: 0.893, Val Accuracy: 0.620, Test Accuracy: 0.606
[7], Best Model with a Train Accuracy: 0.921, Val Accuracy: 0.640, Test Accuracy: 0.627
[8], Best Model with a Train Accuracy: 0.950, Val Accuracy: 0.662, Test Accuracy: 0.676
[9], Best Model with a Train Accuracy: 0.964, Val Accuracy: 0.680, Test Accuracy: 0.671
[10], Best Model with a Train Accurac

### Graph Neural ODE

In [25]:
model_ode_name = 'graph_neural_ode'
graph_neural_ode = initialize_graph_neural_ode

acc_ode = evaluate_uncertainty_quantification(graph_neural_ode, num_experiments, model_ode_name, entropy_thresholds, data, model_type='deterministic', is_stochastic=False)
save_results(acc_ode, model_ode_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.157, Val Accuracy: 0.078, Test Accuracy: 0.093
[0], Loss: 1.948, Train Accuracy: 0.157, Val Accuracy: 0.078, Test Accuracy: 0.093
[1], Best Model with a Train Accuracy: 0.243, Val Accuracy: 0.154, Test Accuracy: 0.183
[2], Best Model with a Train Accuracy: 0.393, Val Accuracy: 0.296, Test Accuracy: 0.330


/tmp/ipykernel_3043090/2244855087.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation(x)


[9], Best Model with a Train Accuracy: 0.629, Val Accuracy: 0.354, Test Accuracy: 0.388
[10], Best Model with a Train Accuracy: 0.664, Val Accuracy: 0.416, Test Accuracy: 0.447
[11], Best Model with a Train Accuracy: 0.686, Val Accuracy: 0.492, Test Accuracy: 0.520


[12], Best Model with a Train Accuracy: 0.671, Val Accuracy: 0.538, Test Accuracy: 0.564
[13], Best Model with a Train Accuracy: 0.686, Val Accuracy: 0.546, Test Accuracy: 0.583
[19], Best Model with a Train Accuracy: 0.843, Val Accuracy: 0.592, Test Accuracy: 0.628
[20], Best Model with a Train Accuracy: 0.900, Val Accuracy: 0.640, Test Accuracy: 0.669
[21], Best Model with a Train Accuracy: 0.921, Val Accuracy: 0.680, Test Accuracy: 0.700
[22], Best Model with a Train Accuracy: 0.914, Val Accuracy: 0.702, Test Accuracy: 0.726
[23], Best Model with a Train Accuracy: 0.914, Val Accuracy: 0.724, Test Accuracy: 0.761
[24], Best Model with a Train Accuracy: 0.929, Val Accuracy: 0.738, Test Accuracy: 0.776
[25], Best Model with a Train Accuracy: 0.943, Val Accuracy: 0.754, Test Accuracy: 0.787
[26], Best Model with a Train Accuracy: 0.964, Val Accuracy: 0.760, Test Accuracy: 0.785
[27], Best Model with a Train Accuracy: 0.957, Val Accuracy: 0.762, Test Accuracy: 0.778
[30], Best Model with

### GCN

In [26]:
initialize_gcn_model()

MyGCN2(
  (conv1): GCNConv(1433, 64)
  (conv4): GCNConv(64, 7)
)

In [27]:
model_gcn_name = 'gcn'
gcn_model = initialize_gcn_model

acc_gcn = evaluate_uncertainty_quantification(gcn_model, num_experiments, model_gcn_name, entropy_thresholds, data, model_type='deterministic', is_stochastic=False)
save_results(acc_gcn, model_gcn_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.150, Val Accuracy: 0.058, Test Accuracy: 0.066
[0], Loss: 1.956, Train Accuracy: 0.150, Val Accuracy: 0.058, Test Accuracy: 0.066
[2], Best Model with a Train Accuracy: 0.157, Val Accuracy: 0.070, Test Accuracy: 0.074
[3], Best Model with a Train Accuracy: 0.214, Val Accuracy: 0.114, Test Accuracy: 0.136
[4], Best Model with a Train Accuracy: 0.236, Val Accuracy: 0.166, Test Accuracy: 0.196
[5], Best Model with a Train Accuracy: 0.250, Val Accuracy: 0.226, Test Accuracy: 0.248
[6], Best Model with a Train Accuracy: 0.286, Val Accuracy: 0.258, Test Accuracy: 0.292
[7], Best Model with a Train Accuracy: 0.286, Val Accuracy: 0.272, Test Accuracy: 0.310
[8], Best Model with a Train Accuracy: 0.286, Val Accuracy: 0.282, Test Accuracy: 0.322
[9], Best Model with a Train Accuracy: 0.286, Val Accuracy: 0.292, Test Accuracy: 0.326
[12], Best Model with a Train Accuracy: 0.307, Val Accuracy: 0.294, Test Accuracy: 0.331
[13], Best Model

[36], Best Model with a Train Accuracy: 0.600, Val Accuracy: 0.542, Test Accuracy: 0.559
[37], Best Model with a Train Accuracy: 0.607, Val Accuracy: 0.548, Test Accuracy: 0.563
[39], Best Model with a Train Accuracy: 0.657, Val Accuracy: 0.566, Test Accuracy: 0.585
[40], Best Model with a Train Accuracy: 0.671, Val Accuracy: 0.588, Test Accuracy: 0.595
[41], Best Model with a Train Accuracy: 0.686, Val Accuracy: 0.598, Test Accuracy: 0.608
[42], Best Model with a Train Accuracy: 0.700, Val Accuracy: 0.618, Test Accuracy: 0.623
[43], Best Model with a Train Accuracy: 0.714, Val Accuracy: 0.628, Test Accuracy: 0.637
[44], Best Model with a Train Accuracy: 0.729, Val Accuracy: 0.634, Test Accuracy: 0.655
[45], Best Model with a Train Accuracy: 0.743, Val Accuracy: 0.660, Test Accuracy: 0.673
[46], Best Model with a Train Accuracy: 0.771, Val Accuracy: 0.674, Test Accuracy: 0.685
[47], Best Model with a Train Accuracy: 0.800, Val Accuracy: 0.680, Test Accuracy: 0.697
[48], Best Model with

### Bayesian GCN

In [28]:
model_bayesian_gcn_name = 'bayesian_gcn'
bayesian_gcn_model = initialize_bayesian_gcn_model


acc_bayesian_gcn = evaluate_uncertainty_quantification(bayesian_gcn_model, num_experiments, model_bayesian_gcn_name, entropy_thresholds, data, model_type='stochastic', is_stochastic=True)
save_results(acc_bayesian_gcn, model_bayesian_gcn_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.164, Val Accuracy: 0.086, Test Accuracy: 0.095
[0], Loss: 1.999, Train Accuracy: 0.164, Val Accuracy: 0.086, Test Accuracy: 0.095
[1], Best Model with a Train Accuracy: 0.143, Val Accuracy: 0.162, Test Accuracy: 0.146
[5], Best Model with a Train Accuracy: 0.214, Val Accuracy: 0.212, Test Accuracy: 0.179
[15], Best Model with a Train Accuracy: 0.393, Val Accuracy: 0.256, Test Accuracy: 0.304
[20], Best Model with a Train Accuracy: 0.464, Val Accuracy: 0.272, Test Accuracy: 0.328
[28], Best Model with a Train Accuracy: 0.407, Val Accuracy: 0.276, Test Accuracy: 0.268


[30], Best Model with a Train Accuracy: 0.464, Val Accuracy: 0.290, Test Accuracy: 0.300
[37], Best Model with a Train Accuracy: 0.629, Val Accuracy: 0.370, Test Accuracy: 0.426
[42], Best Model with a Train Accuracy: 0.557, Val Accuracy: 0.406, Test Accuracy: 0.449
[47], Best Model with a Train Accuracy: 0.707, Val Accuracy: 0.574, Test Accuracy: 0.592
[53], Best Model with a Train Accuracy: 0.807, Val Accuracy: 0.594, Test Accuracy: 0.610
[62], Best Model with a Train Accuracy: 0.850, Val Accuracy: 0.626, Test Accuracy: 0.649
[71], Best Model with a Train Accuracy: 0.807, Val Accuracy: 0.666, Test Accuracy: 0.689
[72], Best Model with a Train Accuracy: 0.836, Val Accuracy: 0.674, Test Accuracy: 0.686
[80], Best Model with a Train Accuracy: 0.850, Val Accuracy: 0.676, Test Accuracy: 0.694
[84], Best Model with a Train Accuracy: 0.843, Val Accuracy: 0.686, Test Accuracy: 0.691
[86], Best Model with a Train Accuracy: 0.843, Val Accuracy: 0.706, Test Accuracy: 0.705
[94], Best Model with

### Ensemble of GCN

In [29]:
model_ensemble = 'ensemble_gcn'

ensemble_acc = evaluate_uncertainty_quantification_ensemble(num_experiments, model_ensemble, entropy_thresholds, data)
save_results(ensemble_acc, model_ensemble, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.143, Val Accuracy: 0.156, Test Accuracy: 0.144
[0], Loss: 1.960, Train Accuracy: 0.143, Val Accuracy: 0.156, Test Accuracy: 0.144
[1], Best Model with a Train Accuracy: 0.150, Val Accuracy: 0.160, Test Accuracy: 0.145
[2], Best Model with a Train Accuracy: 0.229, Val Accuracy: 0.208, Test Accuracy: 0.208
[3], Best Model with a Train Accuracy: 0.279, Val Accuracy: 0.244, Test Accuracy: 0.251
[20], Best Model with a Train Accuracy: 0.557, Val Accuracy: 0.260, Test Accuracy: 0.288
[21], Best Model with a Train Accuracy: 0.600, Val Accuracy: 0.282, Test Accuracy: 0.310
[22], Best Model with a Train Accuracy: 0.643, Val Accuracy: 0.294, Test Accuracy: 0.329
[23], Best Model with a Train Accuracy: 0.686, Val Accuracy: 0.326, Test Accuracy: 0.350
[24], Best Model with a Train Accuracy: 0.700, Val Accuracy: 0.352, Test Accuracy: 0.371
[25], Best Model with a Train Accuracy: 0.714, Val Accuracy: 0.360, Test Accuracy: 0.393
[26], Best 

[36], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.550, Test Accuracy: 0.599
[37], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.562, Test Accuracy: 0.608
[38], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.574, Test Accuracy: 0.618
[39], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.582, Test Accuracy: 0.625
[40], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.588, Test Accuracy: 0.636
[41], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.592, Test Accuracy: 0.638
[42], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.596, Test Accuracy: 0.644
[43], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.602, Test Accuracy: 0.644
[44], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.606, Test Accuracy: 0.647
[45], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.624, Test Accuracy: 0.652
[46], Best Model with a Train Accuracy: 0.829, Val Accuracy: 0.630, Test Accuracy: 0.654
[47], Best Model with